In [1]:
from data_keeper import get_data_keeper

In [2]:
from testing import test_models_with_drugs

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV

In [4]:
get_data_keeper().get_possible_drugs()

[u'ETHA: Ethambutol ',
 u'ISON: Isoniazid ',
 u'RIFM: rifampicin ',
 u'RIFP: Rifapentine ',
 u'PYRA: Pyrazinamide ',
 u'STRE: Streptomycin ',
 u'CYCL: Cycloserine ',
 u'ETHI: Ethionamide/ Prothionamide ',
 u'PARA: Para-aminosalicyclic acid ',
 u'AMIK: Amikacin',
 u' Kanamycin ',
 u'CAPR: Capreomycin ',
 u'OFLO: Ofloxacin ']

In [7]:
from wrappers import GridSearchCVWrapper
from wrappers import XGBoostClassifierFeatureImportances as XGB
from wrappers import MatrixCleanerWrapper
from wrappers import SparseWrapper
from wrappers import ModelFeatureSelectionWrapper

ImportError: cannot import name MatrixCleanerWrapper

In [45]:
class ModelBasedFeatureImportanceGetter(BaseEstimator):
    def __init__(self, inner_model):
        self.inner_model = inner_model
    
    def get_feature_importances(self, X, y):
        return self.inner_model.fit(X, y).feature_importances_

In [8]:
from frn import FeatureRelevanceNetwork

In [47]:
def get_complete_linear_model():
    inner_model = LogisticRegressionCV(Cs=[10 ** i for i in xrange(-4, 4)], solver='liblinear')
    outer_model = GridSearchCV(inner_model, {'penalty': ['l1', 'l2']})
    return MatrixCleaningWrapper(SparseWrapper(outer_model))

In [55]:
def get_complete_tree_based_model():
    cv_params = {'inner_model__inner_model__n_estimators': [1],#, 5, 10, 20, 50, 100], 
                 'feature_selection_threshold_coef': [0.1]}#, 1, 3, 10, 30, 100, 300]}
    return MatrixCleaningWrapper(FeatureRelevanceNetwork(SparseWrapper(XGBoostClassifierFeatureImportances()), ModelBasedFeatureImportanceGetter(SparseWrapper(XGBoostClassifierFeatureImportances(n_estimators=10)))))

In [56]:
model = get_complete_tree_based_model()
test_models_with_drugs([('xgboost', model)], [u'ETHI: Ethionamide/ Prothionamide '], metrics=['accuracy', 'confusion matrix', 'feature count'])

cleaner (122, 2718)
{'f1401': 1, 'f1060': 1, 'f606': 3, 'f202': 3, 'f210': 2, 'f2193': 1, 'f160': 1, 'f2439': 3, 'f2168': 5, 'f970': 1, 'f458': 8, 'f2586': 2, 'f996': 8, 'f2161': 2, 'f504': 1}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2705)
{'f2157': 1, 'f198': 5, 'f2034': 1, 'f2411': 1, 'f1954': 1, 'f1499': 1, 'f95': 2, 'f994': 9, 'f995': 1, 'f2418': 5, 'f463': 10, 'f1389': 2}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2770)
{'f1417': 4, 'f630': 4, 'f2230': 2, 'f208': 1, 'f2479': 4, 'f1384': 1, 'f1011': 10, 'f99': 4, 'f476': 10}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2712)
{'f204': 3, 'f2169': 2, 'f2153': 1, 'f466': 10, 'f1384': 1, 'f96': 5, 'f2421': 4, 'f989': 10, 'f2360': 2, 'f45': 1, 'f619': 3, 'f1389': 1}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2852)
{'f1049': 10, 'f215': 5, 'f2551': 3, 'f2142': 2, 'f2274': 3, 'f490': 10, 'f2729': 2, 'f101': 2, 'f649': 1}
<class 'scipy.sparse.csr.csr_matrix'>
cleaner (122, 2692)
{'f452': 10, 'f156': 1, '

{('xgboost',
  u'ETHI: Ethionamide/ Prothionamide '): {'accuracy': 0.84558823529411764, 'confusion matrix': array([[94,  6],
         [15, 21]]), 'feature count': 12}}

In [16]:
X = get_data_keeper().get_common_x()

In [10]:
from scipy.stats import norm
def results_differ_p_value(y_true, y1, y2):
    y1 = (np.array(y1) == np.array(y_true)).astype(np.float64)
    y2 = (np.array(y2) == np.array(y_true)).astype(np.float64)
    diff = y1 - y2
    norm_stat = diff.mean() / diff.std() * np.sqrt(diff.shape[0])
    quantile = norm.cdf(norm_stat)
    return min(quantile, 1.0 - quantile)

In [11]:
results_differ_p_value([0] * 100 + [1] * 32, [0] * 100 + [1] * 32, [0] * 99 + [1] * 32 + [0])

0.077071286548999973